In [43]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


In [44]:
df = pd.read_csv("NYC_vehicle_crash.csv", low_memory = False)


In [45]:
# find and fill zipcode/Borough location based on lattitude and longtide data 
# find  location

# specific month had more accident than other? line graph by month; find possible reasons 
# specific hour ? : hist by hour 
# which borough /zipcode : hist ; % of accident per 100 resident 
# geo plot lattitude, longitude 
# contributing factors : unique, sort_values(desc): hist, 
# is coupe (any specific vehicle) more inclined to crash? pie-chart 
# how maany injured by year/month  # if changed by year, is there any 
# how many killed 
# hypothesis testing? or statistics : mean, curve shape, distribution 

# conclusions: top reasons of crashes and  ideas/regulations to stop them 
#   - specific hour, month, locations: so need more petrolling, or traffic signs, lights 

In [46]:
# df.head(1)
# df.shape
# df.info()
# df.columns
# df.describe()
df.head()


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,7/26/18,12:10,NaN,NaN,40.704494,-73.81743,POINT (-73.81743 40.704494),VAN WYCK EXPWY,NaN,NaN,...,Unspecified,Unspecified,NaN,NaN,3948740,Sedan,Sedan,Taxi,NaN,NaN
1,7/12/18,17:40,BRONX,10470,40.903890,-73.85139,POINT (-73.85139 40.90389),EAST 241 STREET,RICHARDSON AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,3943762,Sedan,Sedan,NaN,NaN,NaN
2,7/11/18,16:30,MANHATTAN,10022,40.760464,-73.97372,POINT (-73.97372 40.760464),NaN,NaN,532 MADISON AVENUE,...,Passing or Lane Usage Improper,NaN,NaN,NaN,3939841,Van,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,7/6/18,19:40,BRONX,10469,40.872826,-73.85210,POINT (-73.8521 40.872826),NaN,NaN,1215 EAST GUN HILL ROAD,...,Unsafe Speed,Unspecified,NaN,NaN,3936658,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN
4,7/26/18,19:13,BRONX,10469,40.866510,-73.85772,POINT (-73.85772 40.86651),NaN,NaN,2789 HONE AVENUE,...,Other Vehicular,NaN,NaN,NaN,3949531,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [15]:
# To use as a single arguement of form (lat, long) in funcion, create and format a column in the form of lat, long
df['lattitude_str'] = df['LATITUDE'].astype(str)

df['longitude_str'] = df['LONGITUDE'].astype(str)

df['coord_point'] = df['lattitude_str'].str.cat(df['longitude_str'], sep = ',')

In [16]:
#df.BOROUGH.unique()
df['coord_point'].unique()
# df.groupby('CONTRIBUTING FACTOR VEHICLE 1').count().sort_values(by ='CONTRIBUTING FACTOR VEHICLE 1', ascending = False )


array(['40.704494,-73.81743', '40.903890000000004,-73.85139000000001',
       '40.760464,-73.97372', ..., '40.6652473,-73.9923151',
       '40.6071047,-74.0024094', '40.8300385,-73.9366228'], dtype=object)

In [17]:
# df['zip_code'] = df['LOCATION'].apply(find_zip_from_lati_longitude)

In [18]:
# check how many null/missing values on columns 
count_zip_Nan = df["ZIP CODE"].isna().sum() # 348940
count_zip_Nan
count_borough_Nan = df["BOROUGH"].isna().sum() # 348757
count_borough_Nan
count_latt_Nan = df['LATITUDE'].isna().sum()  # 110155
count_on_street_Nan = df['ON STREET NAME'].isna().sum() # 230393
count_on_street_Nan
count_cstreet_Nan = df['CROSS STREET NAME'].isna().sum() # 443021
count_cstreet_Nan


443021

In [19]:
# main data to work on 
crash = df[['COLLISION_ID', 'CRASH DATE', 'CRASH TIME' ,'BOROUGH', 'ZIP CODE', 'CONTRIBUTING FACTOR VEHICLE 1',
      'CONTRIBUTING FACTOR VEHICLE 2', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'coord_point']]



In [20]:
crash.head(2)

,COLLISION_ID,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,coord_point
0,3948740,7/26/18,12:10,NaN,NaN,Unsafe Lane Changing,Unspecified,Sedan,Sedan,"40.704494,-73.81743"
1,3943762,7/12/18,17:40,BRONX,10470,Following Too Closely,Unspecified,Sedan,Sedan,"40.903890000000004,-73.85139000000001"


In [21]:
#%load_ext autotime

In [24]:
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'my_app/1'
geopy.geocoders.options.default_timeout = 7
geolocator = Nominatim()
print(geolocator.headers)
{'User-Agent': 'my_app/1'}
print(geolocator.timeout)

{'User-Agent': 'my_app/1'}
7


In [36]:
# for lattitude/longtide to borough/zip code conversion 

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
geopy.geocoders.options.default_timeout = 7

def find_zipcode_from_lati_longitude(coordinates):
    # if lattitude = nan or longitude = nan,return (00000)
    '''
    Given a lattitude and longtitude coordinates (from Pandas Dataframe, df,)
    return zipcode of that coordinates
    
    Parameter
    ----------
    coordinates: point with x,y in the form: lattitude, longitude 
    
    Returns:
    --------
    zipcode: (int) zipcode of coordinates 
    '''
    if coordinates == 'nan,nan':
        zipcode = '00000'
    else: 
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.reverse(coordinates)
        #location.raw
        full_address = str(location.address)
        lst_of_address = full_address.split(',')
        zipcode = lst_of_address[-2]

    return zipcode #, borough

coordinates = df.loc[7, 'coord_point']

find_zipcode_from_lati_longitude(coordinates)

'00000'

In [39]:

def find_borough_from_lati_longitude(coordinates):
    # if lattitude = nan or longitude = nan,return 'Not available'

    '''
    Given a lattitude and longtitude coordinates (from Pandas Dataframe, df,)
    return city-borough of that coordinates
    
    Parameter
    ----------
    coordinates: point with x,y in the form: lattitude, longitude 
    
    Returns:
    --------
    borough: (str) borough(eg. Queens, Brooklyn, etc.) of coordinates 
    '''
    if coordinates == 'nan,nan':
        borough = 'Not availbe'
    else: 
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.reverse(coordinates)
        #location.raw
        full_address = str(location.address)
        lst_of_address = full_address.split(',')
        borough = lst_of_address[-5]
        #zipcode = lst_of_address[-2]

    return borough 

coordinates = df.loc[7, 'coord_point']

cr = df['coord_point'][343]

find_borough_from_lati_longitude(cr)

' Brooklyn'

In [41]:
# use small data to test if function works 

df1 = df.head(10).copy()
df1.shape


df1['zip_code'] = df1['coord_point'].apply(lambda x:find_zipcode_from_lati_longitude(x))
df1['NYC_borough'] = df1['coord_point'].apply(lambda x:find_borough_from_lati_longitude(x))


In [42]:
df1.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,lattitude_str,longitude_str,coord_point,month,zip_code,NYC_borough
0,7/26/18,12:10,NaN,NaN,40.704494,-73.81743,POINT (-73.81743 40.704494),VAN WYCK EXPWY,NaN,NaN,...,Sedan,Taxi,NaN,NaN,40.704494,-73.81743,"40.704494,-73.81743",2018-07,11435,Queens
1,7/12/18,17:40,BRONX,10470,40.903890,-73.85139,POINT (-73.85139 40.90389),EAST 241 STREET,RICHARDSON AVENUE,NaN,...,Sedan,NaN,NaN,NaN,40.903890000000004,-73.85139000000001,"40.903890000000004,-73.85139000000001",2018-07,10470,The Bronx
2,7/11/18,16:30,MANHATTAN,10022,40.760464,-73.97372,POINT (-73.97372 40.760464),NaN,NaN,532 MADISON AVENUE,...,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,40.760464,-73.97372,"40.760464,-73.97372",2018-07,10022,Manhattan
3,7/6/18,19:40,BRONX,10469,40.872826,-73.85210,POINT (-73.8521 40.872826),NaN,NaN,1215 EAST GUN HILL ROAD,...,Sedan,Sedan,NaN,NaN,40.872826,-73.8521,"40.872826,-73.8521",2018-07,10469,The Bronx
4,7/26/18,19:13,BRONX,10469,40.866510,-73.85772,POINT (-73.85772 40.86651),NaN,NaN,2789 HONE AVENUE,...,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,40.86651,-73.85772,"40.86651,-73.85772",2018-07,10469,The Bronx


In [47]:
# extract months to compare and plot monthly car crash 
df['month'] = pd.to_datetime(df['CRASH DATE']).dt.to_period('M')

df['month'].unique()


<PeriodArray>
['2018-07', '2016-04', '2020-03', '2015-05', '2015-03', '2020-01', '2019-12',
 '2016-05', '2016-06', '2018-06', '2020-02', '2018-03', '2018-04', '2017-10',
 '2017-11', '2015-06', '2018-05', '2019-11', '2019-09', '2016-03', '2014-12',
 '2015-01', '2019-10', '2014-11', '2018-02', '2018-01', '2015-02', '2019-08',
 '2017-09', '2017-08', '2019-07', '2014-09', '2014-10', '2014-08', '2019-06',
 '2014-01', '2016-02', '2019-05', '2013-12', '2016-01', '2019-04', '2017-12',
 '2013-11', '2015-12', '2019-03', '2019-02', '2014-07', '2014-06', '2014-05',
 '2019-01', '2018-12', '2013-01', '2012-12', '2018-11', '2014-04', '2014-03',
 '2018-10', '2014-02', '2018-09', '2015-04', '2012-11', '2018-08', '2015-11',
 '2015-10', '2015-09', '2017-07', '2017-06', '2017-05', '2013-10', '2017-04',
 '2015-08', '2013-09', '2015-07', '2013-08', '2017-03', '2017-02', '2017-01',
 '2016-12', '2016-11', '2016-10', '2012-10', '2016-09', '2012-09']
Length: 83, dtype: period[M]

In [32]:
 df.sort_values(by = 'month', ascending=False).groupby('month').head(3)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,lattitude_str,longitude_str,coord_point,month
46224,3/7/20,20:00,BROOKLYN,11207,40.663520,-73.886770,POINT (-73.88677 40.66352),NaN,NaN,727 HENDRIX STREET,...,4305793,Sedan,NaN,NaN,NaN,NaN,40.66352,-73.88677,"40.66352,-73.88677",2020-03
55388,3/1/20,9:10,MANHATTAN,10016,40.746883,-73.982925,POINT (-73.982925 40.746883),NaN,NaN,45 EAST 33 STREET,...,4296316,Pick-up Truck,Sedan,NaN,NaN,NaN,40.746883000000004,-73.982925,"40.746883000000004,-73.982925",2020-03
55375,3/19/20,17:15,NaN,NaN,40.698837,-73.914070,POINT (-73.91407 40.698837),LINDEN STREET,NaN,NaN,...,4303218,Pick-up Truck,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,40.698837,-73.91407,"40.698837,-73.91407",2020-03
62415,2/7/20,16:30,NaN,NaN,40.788784,-73.791160,POINT (-73.79116 40.788784),CLEARVIEW EXPRESSWAY,NaN,NaN,...,4285032,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,40.788784,-73.79115999999999,"40.788784,-73.79115999999999",2020-02
61928,2/10/20,10:00,BROOKLYN,11231,40.680630,-73.996730,POINT (-73.99673 40.68063),NaN,NaN,375 COURT STREET,...,4286673,Taxi,Sedan,NaN,NaN,NaN,40.68063,-73.99673,"40.68063,-73.99673",2020-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040091,10/12/12,23:15,NaN,NaN,NaN,NaN,NaN,EAST FORDHAM ROAD,EAST KINGSBRIDGE ROAD,NaN,...,110133,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,nan,nan,"nan,nan",2012-10
1039474,10/9/12,12:07,BRONX,10453,40.853815,-73.907331,POINT (-73.9073312 40.8538153),EAST BURNSIDE AVENUE,JEROME AVENUE,NaN,...,92526,UNKNOWN,NaN,NaN,NaN,NaN,40.8538153,-73.9073312,"40.8538153,-73.9073312",2012-10
1047952,9/28/12,10:30,NaN,NaN,40.847625,-73.836332,POINT (-73.836332 40.8476255),NaN,NaN,NaN,...,2912432,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,40.8476255,-73.836332,"40.8476255,-73.836332",2012-09
1046913,9/28/12,10:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2864223,UNKNOWN,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,nan,nan,"nan,nan",2012-09


In [252]:
# modify from above to use lattitude, longitude inplace of coordinates 
def find_zip_from_lati_points(latt, longt):
#def find_zip_from_lati_longitude(coordinates):

    
    locator = Nominatim(user_agent="myGeocoder")
    location = locator.reverse(latt, longt)
    location.raw
    full_address = location.address
    lst_of_address = full_address.split(',')
    zipcode = lst_of_address[5]
    
    return zipcode

In [49]:

find_zip_from_lati_points(40.903890, -73.85139)

' España'

In [50]:
type(cr)

str

In [55]:
#df['borough'] = df.apply(find_borough_from_lati_longitude(coordinates), args=df['new_col'])

In [14]:
df[['LONGITUDE']].head()

,LONGITUDE
0,-73.81743
1,-73.85139
2,-73.97372
3,-73.85210
4,-73.85772


time: 23.9 ms


In [15]:
df[['LONGITUDE']]

,LONGITUDE
0,-73.817430
1,-73.851390
2,-73.973720
3,-73.852100
4,-73.857720
...,...
1048570,-73.995109
1048571,-74.092618
1048572,NaN
1048573,NaN


time: 22.4 ms


In [58]:
# make tuple from long/latt
df['zip_tuple'] = list(zip(df.LONGITUDE, df.LONGITUDE))

In [59]:
df['new_col']

0                            (-73.81743, -73.81743)
1          (-73.85139000000001, -73.85139000000001)
2                            (-73.97372, -73.97372)
3                              (-73.8521, -73.8521)
4                            (-73.85772, -73.85772)
                             ...                   
1048570                  (-73.9951087, -73.9951087)
1048571                  (-74.0926175, -74.0926175)
1048572                                  (nan, nan)
1048573                                  (nan, nan)
1048574                  (-73.8667311, -73.8667311)
Name: new_col, Length: 1048575, dtype: object

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 34 columns):
CRASH DATE                       1048575 non-null object
CRASH TIME                       1048575 non-null object
BOROUGH                          699818 non-null object
ZIP CODE                         699635 non-null object
LATITUDE                         938420 non-null float64
LONGITUDE                        938420 non-null float64
LOCATION                         938420 non-null object
ON STREET NAME                   818182 non-null object
CROSS STREET NAME                605554 non-null object
OFF STREET NAME                  198533 non-null object
NUMBER OF PERSONS INJURED        1048558 non-null float64
NUMBER OF PERSONS KILLED         1048544 non-null float64
NUMBER OF PEDESTRIANS INJURED    1048575 non-null int64
NUMBER OF PEDESTRIANS KILLED     1048575 non-null int64
NUMBER OF CYCLIST INJURED        1048575 non-null int64
NUMBER OF CYCLIST KILLED         104857

In [17]:
# ll = df[[str('LONGITUDE')]]+df[[str('LONGITUDE')]]
# ll

time: 264 µs


In [19]:
# fill missing "BOROUGH" and "ZIP CODE" with appropriate value from location 
# use pgeocode ; no method to find latt,longt to borough :: check reverse_geocoder 

# import pgeocode 
# usa = pgeocode.Nominatim("us")
# ciy = usa.query_postal_code('11373') 
# xxx

time: 346 µs


In [ ]:
# from pygeocoder import Geocoder
# results = Geocoder.reverse_geocode(31, 87)

In [ ]:
# for geo map find lattitude and longitude 
geo_loc = df[['LATITUDE', 'LONGITUDE']]
# geo_loc

In [ ]:
# map hist/pie chart by zip code : 430 zip codes, no feasilbe probaboly?
by_zipcode = df[['ZIP CODE', 'COLLISION_ID']].groupby('ZIP CODE').count()

# max or top 10 zip code for crashes 
by_zipcode.sort_values(by = 'COLLISION_ID', ascending = False ).head(10)

In [ ]:
type(by_zipcode)